# 📝 Demo: Phân loại cảm xúc với Machine Learning  

## 🔍 Giới thiệu  

Trong bài toán **phân loại cảm xúc (Sentiment Classification)**, chúng ta cần xác định một đánh giá là **tích cực (positive)** hay **tiêu cực (negative)**.  

Bài toán này có nhiều ứng dụng thực tế, bao gồm:  
✅ Phân tích phản hồi khách hàng (Shopee, Tiki, Amazon).  
✅ Xây dựng chatbot hỗ trợ khách hàng.  
✅ Giám sát thương hiệu trên mạng xã hội.  

Mô hình được huấn luyện từ **tập dữ liệu đánh giá sản phẩm**, sau đó có thể dự đoán cảm xúc của **câu mới** nhập vào hệ thống.  

---

## 🏗️ Mô hình sử dụng trong demo  

### 🔹 **1. Trích xuất đặc trưng từ văn bản**  
📌 Dùng **TF-IDF (Term Frequency - Inverse Document Frequency)** để biến văn bản thành vector số.  
- **TF** đo tần suất xuất hiện của một từ trong văn bản.  
- **IDF** giúp giảm trọng số của các từ xuất hiện quá nhiều trong nhiều tài liệu.  
- **TF-IDF(t, d) = TF(t, d) × IDF(t)**  

---

### 🔹 **2. Chọn lọc đặc trưng quan trọng**  
📌 Sử dụng **Chi-square Test ($\chi^2$)** để chọn **50 đặc trưng quan trọng nhất** giúp phân biệt **tích cực và tiêu cực**.  
- Đánh giá xem từ nào xuất hiện **nhiều trong đánh giá tích cực/tiêu cực**.  

---

### 🔹 **3. Huấn luyện mô hình phân loại**  
📌 Dùng **Multinomial Naïve Bayes (MNB)** để huấn luyện mô hình.  
- **Naïve Bayes giả định rằng các từ trong văn bản là độc lập**.  
- Xác suất của một lớp $C$ (tích cực/tiêu cực) được tính bằng:  
  $$ P(C|X) = \frac{P(X|C) \cdot P(C)}{P(X)} $$  
- **MNB phù hợp với dữ liệu dạng phân phối xác suất rời rạc**, như tần suất xuất hiện từ trong văn bản.  

---

### 🔹 **4. Dự đoán cảm xúc câu mới**  
📌 Khi nhập một câu mới, hệ thống sẽ:  
- **Chuyển câu thành vector TF-IDF**.  
- **Giữ lại các đặc trưng quan trọng nhất**.  
- **Dùng mô hình Naïve Bayes để dự đoán** cảm xúc.  

Ví dụ:  
```python
search_engine("màu sắc không giống trong ảnh") 
# Kết quả: "Tiêu cực"

In [2]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## 🔎 1. Dữ liệu đầu vào (Input Data)  
- **Tập văn bản (documents)** gồm **30 câu đánh giá** về sản phẩm/dịch vụ.  
- **Nhãn (labels)**:  
  - `1` = Tích cực (Positive)  
  - `0` = Tiêu cực (Negative)  

Ví dụ dữ liệu:  
- `"sản phẩm này rất tốt và hữu ích"` → **1 (tích cực)**  
- `"dịch vụ tệ, không bao giờ quay lại"` → **0 (tiêu cực)**  

In [3]:
# 1. Dữ liệu mẫu: thêm nhiều tài liệu hơn
documents = [
    "sản phẩm này rất tốt và hữu ích",
    "tôi không thích dịch vụ này chút nào",
    "chất lượng tuyệt vời, giao hàng nhanh",
    "tệ quá, không đáng tiền",
    "trải nghiệm tuyệt vời, sẽ quay lại",
    "dịch vụ kém, thái độ không tốt",
    "hàng đẹp, giá cả hợp lý",
    "giao hàng chậm, sản phẩm lỗi",
    "rất hài lòng với chất lượng",
    "không bao giờ mua lại, quá tệ",
    "sản phẩm rất tuyệt, tôi sẽ giới thiệu cho bạn bè",
    "quá thất vọng, hàng không giống mô tả",
    "dịch vụ khách hàng chu đáo, rất hài lòng",
    "hàng kém chất lượng, bị hỏng ngay khi mở hộp",
    "tôi cực kỳ thích sản phẩm này, chất lượng vượt mong đợi",
    "hàng giao sai màu, không đáng tiền",
    "mua lần thứ hai, vẫn rất hài lòng",
    "hàng bị móp méo, đóng gói quá kém",
    "giao nhanh, hàng đúng như mô tả, rất ưng ý",
    "dịch vụ tệ, không bao giờ quay lại",
    "đóng gói cẩn thận, sản phẩm đúng như mô tả",
    "sản phẩm bị lỗi ngay khi mở hộp, quá thất vọng",
    "giá cả hợp lý, chất lượng trên cả mong đợi",
    "tôi đã đặt hàng nhưng chưa nhận được, quá chậm",
    "hàng chính hãng, rất đáng tiền",
    "sản phẩm không hoạt động, dịch vụ hỗ trợ kém",
    "rất thích, sẽ tiếp tục ủng hộ",
    "màu sắc không giống trong ảnh, không hài lòng",
    "giao hàng nhanh, đóng gói đẹp, chất lượng tốt",
    "dịch vụ quá tệ, không đáng mua"
]
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # 1: tích cực, 0: tiêu cực

## 🔎 2. Chuyển đổi văn bản thành đặc trưng số (TF-IDF)  
**TF-IDF (Term Frequency - Inverse Document Frequency)** giúp biểu diễn văn bản thành vector số:  

- **TF (Tần suất xuất hiện từ trong văn bản)**  
- **IDF (Độ quan trọng của từ trên toàn bộ tập dữ liệu)**  

Công thức tính:  
$$ TF(t, d) = \frac{\text{số lần xuất hiện của } t \text{ trong } d}{\text{tổng số từ trong } d} $$  
$$ IDF(t) = \log \frac{\text{Tổng số văn bản}}{\text{số văn bản chứa } t} $$  
$$ TF-IDF(t, d) = TF(t, d) \times IDF(t) $$  

In [4]:
# 2. Chuyển đổi văn bản thành đặc trưng số (TF-IDF)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents).toarray()
feature_names = vectorizer.get_feature_names_out()

print("Số lượng đặc trưng ban đầu:", X.shape[1])

Số lượng đặc trưng ban đầu: 106


## 🔎 3. Chọn lọc đặc trưng (Feature Selection - Chi-square Test)  
### **Mục tiêu**:  
- Chỉ giữ lại **50 từ quan trọng nhất** giúp phân biệt tích cực/tiêu cực.  

Công thức kiểm định **Chi-square ($\chi^2$ Test)**:  
$$ \chi^2 = \sum \frac{(O_i - E_i)^2}{E_i} $$  
- $O_i$: Giá trị quan sát thực tế.  
- $E_i$: Giá trị kỳ vọng.  

Danh sách các từ quan trọng có thể là:
['tốt', 'hài lòng', 'tệ', 'thích', 'dịch vụ', 'quá', 'đẹp', ...]

In [5]:
# 3. Lựa chọn đặc trưng - chọn 50 đặc trưng tốt nhất
selector = SelectKBest(chi2, k=50)
X_selected = selector.fit_transform(X, labels)

# Lấy danh sách các đặc trưng được chọn
selected_features = feature_names[selector.get_support()]
print("Các đặc trưng được chọn:", selected_features)

Các đặc trưng được chọn: ['bao' 'bị' 'chính' 'chút' 'chất' 'chậm' 'cả' 'dịch' 'giá' 'giống' 'giờ'
 'hãng' 'hộp' 'hợp' 'hữu' 'khi' 'không' 'kém' 'lý' 'lượng' 'lỗi' 'mong'
 'màu' 'méo' 'móp' 'mở' 'ngay' 'nhanh' 'như' 'nào' 'quá' 'rất' 'sai' 'sẽ'
 'thái' 'thất' 'trải' 'tuyệt' 'tệ' 'và' 'vọng' 'với' 'vời' 'vụ' 'ích'
 'đáng' 'đúng' 'đẹp' 'độ' 'đợi']


## 🔎 4. Chia dữ liệu thành tập huấn luyện và kiểm tra  
- **80% dữ liệu để huấn luyện**  
- **20% dữ liệu để kiểm tra**  

Dùng `train_test_split()` để phân chia dữ liệu ngẫu nhiên.  

## 🔎 5. Huấn luyện mô hình Naïve Bayes  
**Naïve Bayes giả định rằng các từ trong văn bản là độc lập**.  
Công thức xác suất Bayes:  
$$ P(C|X) = \frac{P(X|C) \cdot P(C)}{P(X)} $$  
- $P(C|X)$: Xác suất văn bản thuộc lớp C (Tích cực/Tiêu cực).  
- $P(X|C)$: Xác suất các từ xuất hiện trong lớp C.  
- $P(C)$: Xác suất của lớp C trong toàn bộ dữ liệu.  

Dùng `MultinomialNB()` để huấn luyện mô hình.

In [6]:
# 4. Chia dữ liệu thành tập huấn luyện và kiểm tra (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_selected, labels, test_size=0.2, random_state=42)

# 5. Huấn luyện mô hình phân loại Naive Bayes
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

## 🔎 6. Đánh giá mô hình  
Dùng **accuracy_score** để kiểm tra độ chính xác:  
$$ Accuracy = \frac{\text{số dự đoán đúng}}{\text{tổng số dự đoán}} $$  
Nếu `accuracy = 0.84`, nghĩa là mô hình phân loại **đúng 84% trường hợp**.  

In [7]:
# 6. Dự đoán và đánh giá
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Độ chính xác:", accuracy)

Độ chính xác: 0.8333333333333334


## 🔎 7. Xây dựng công cụ tìm kiếm cảm xúc  
- Nhận một câu đánh giá mới.  
- Chuyển thành vector TF-IDF.  
- Dự đoán cảm xúc bằng mô hình Naïve Bayes.  

Ví dụ kiểm tra:  
```python
test_query = "màu sắc không giống trong ảnh"
result = search_engine(test_query)
print(f"Kết quả tìm kiếm cho '{test_query}': {result}")

In [8]:
# 7. Hàm tìm kiếm đơn giản
def search_engine(query):
    query_vec = vectorizer.transform([query]).toarray()
    query_selected = selector.transform(query_vec)
    prediction = classifier.predict(query_selected)
    return "Tích cực" if prediction[0] == 1 else "Tiêu cực"

# Thử nghiệm tìm kiếm
test_query = "màu sắc không giống trong ảnh"
result = search_engine(test_query)
print(f"Kết quả tìm kiếm cho '{test_query}': {result}")
test_query = "sảm phẩn tuyệt vời"
result = search_engine(test_query)
print(f"Kết quả tìm kiếm cho '{test_query}': {result}")

Kết quả tìm kiếm cho 'màu sắc không giống trong ảnh': Tiêu cực
Kết quả tìm kiếm cho 'sảm phẩn tuyệt vời': Tích cực


Kết quả tìm kiếm cho 'màu sắc không giống trong ảnh': Tiêu cực

## 📊 Tóm tắt quy trình tổng thể  

| **Bước** | **Mô tả** |
|----------|----------|
| **1. Dữ liệu đầu vào** | 30 đánh giá sản phẩm (tích cực hoặc tiêu cực) |
| **2. Biểu diễn TF-IDF** | Chuyển văn bản thành ma trận số |
| **3. Chọn lọc đặc trưng** | Dùng $\chi^2$ để lấy 50 đặc trưng quan trọng nhất |
| **4. Chia dữ liệu** | 80% train / 20% test |
| **5. Huấn luyện mô hình** | Sử dụng **Naïve Bayes** để học từ dữ liệu |
| **6. Đánh giá mô hình** | Độ chính xác trên tập kiểm tra |
| **7. Dự đoán cảm xúc câu mới** | Công cụ tìm kiếm đơn giản dự đoán câu là tích cực hay tiêu cực |